# Virtual Linking Bids for Market-Clearing with Non-Merchant Storage

This is the code for the paper "Virtual Linking Bids for Market-Clearing with Non-Merchant Storage". Different types of market clearing setups can be run and compared, including the examples presented in the paper. The code was written for Julia v1.8.

In [ ]:
# Include the relevant packages
using CSV, DataFrames
using JuMP, HiGHS

The different parameters can be updated in the csv files "data_gen.csv" and "data_load.csv" and below:

In [ ]:
## MARKET DATA
# Duration of time periods (in hours):
∆t = 1

## STORAGE DATA
# Storage capacity (MWh)
E = 2.5 
# Initial state of energy (MWh)
E_0 = 0.0 
;

In [ ]:
# Dataframes are created from the csv files for generators:
G_df = CSV.read("data_gen.csv", DataFrame)

In [ ]:
# And loads:
L_df = CSV.read("data_load.csv", DataFrame)

## Ideal Market Clearing

In [ ]:
# Set the level for the storage system at the end of the test period:
E_final = 0.0
;

In [ ]:
# Run the market clearing
include.(("fct_perfect.jl","fct_mc_all.jl","fct_data_all.jl"))
df_MCP = perfect(∆t, E, E_0, E_final, G_df, L_df)

## Split Market Clearing with Level

In [ ]:
# Set the level for the storage system at the end of each market interval:
E_end = [1.0, 0.0]
;

In [ ]:
# Run the market clearing
include.(("fct_split.jl","fct_mc_all.jl","fct_data_all.jl"))
S_end = zeros(maximum(G_df.MI)) # Set the penalty to zero (no penalty)
df_MCS = split(∆t, E, E_0, G_df, L_df, E_end, S_end)

## Split Market Clearing with Penalty

In [ ]:
# Set the penalty for the storage system at the end of each market interval:
S_end = [2, 0]
;

In [ ]:
# Run the market clearing
include.(("fct_split.jl","fct_mc_all.jl","fct_data_all.jl"))
E_end = zeros(maximum(G_df.MI)) # Set the penalty to zero (no penalty)
df_MCS = split(∆t, E, E_0, G_df, L_df, E_end, S_end)

## Market Clearing with Virtual Linking Bids

In [ ]:
# Set the level for the storage system at the end of each market interval:
E_end = [1.0, 0.0]
# Set a discount between 0 and 1 for the value of the stored energy (0: no discount)
discount = 0
;

In [ ]:
# Version with solution multiplicity
include.(("fct_vlb.jl","fct_mc_vlb.jl","fct_stor_val_update.jl","fct_data_all.jl"))
df_VLB = VLB_no_bin(∆t, E, G_df, L_df, E_end, discount)

In [ ]:
# Version without solution multiplicity (binary variables)
include.(("fct_vlb_bin.jl","fct_mc_vlb_bin.jl","fct_mc_vlb_no_bin.jl","fct_stor_val_update.jl","fct_data_all.jl"))
df_VLB_bin = VLB_bin(∆t, E, G_df, L_df, E_end, discount)